In [21]:
import json
import pandas as pd
from scipy.sparse import csr_matrix
import numpy as np

In [22]:
def load(i):
    start = i*1000
    path = "data/mpd.slice." + str(start) + "-" + str(start+999) + ".json"
    data = json.load(open(path,'r'))
    return pd.DataFrame.from_dict(data['playlists'], orient='columns')

In [23]:
# get tracks from playlists 

songPlaylistArray = []
start = 0
while start != 1:
     thisSlice = load(start)
     songPlaylistArray.append(thisSlice)
     start+= 1
train = pd.concat(songPlaylistArray)

In [24]:
train

,name,collaborative,pid,modified_at,num_tracks,num_albums,num_followers,tracks,num_edits,duration_ms,num_artists,description
0,Throwbacks,false,0,1493424000,52,47,1,"[{'pos': 0, 'artist_name': 'Missy Elliott', 't...",6,11532414,37,NaN
1,Awesome Playlist,false,1,1506556800,39,23,1,"[{'pos': 0, 'artist_name': 'Survivor', 'track_...",5,11656470,21,NaN
2,korean,false,2,1505692800,64,51,1,"[{'pos': 0, 'artist_name': 'Hoody', 'track_uri...",18,14039958,31,NaN
3,mat,false,3,1501027200,126,107,1,"[{'pos': 0, 'artist_name': 'Camille Saint-Saën...",4,28926058,86,NaN
4,90s,false,4,1401667200,17,16,2,"[{'pos': 0, 'artist_name': 'The Smashing Pumpk...",7,4335282,16,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
995,old,false,995,1507852800,41,40,1,"[{'pos': 0, 'artist_name': 'Katrina', 'track_u...",8,9917901,36,NaN
996,Daze,false,996,1479254400,17,17,1,"[{'pos': 0, 'artist_name': 'PARTYNEXTDOOR', 't...",13,3699248,15,NaN
997,rap,false,997,1410307200,119,98,1,"[{'pos': 0, 'artist_name': 'LoveRance', 'track...",63,27538723,82,NaN
998,Country,false,998,1507939200,108,75,1,"[{'pos': 0, 'artist_name': 'Hunter Hayes', 'tr...",37,24950143,40,NaN


In [25]:
train.loc[train['pid'] == 844]

,name,collaborative,pid,modified_at,num_tracks,num_albums,num_followers,tracks,num_edits,duration_ms,num_artists,description
844,nostalgia,false,844,1487289600,150,86,1,"[{'pos': 0, 'artist_name': 'The Strokes', 'tra...",12,33568346,55,NaN


In [26]:
# get table where each row represents a song within a playlist

exploded_train = train.explode('tracks')
exploded_train.reset_index(drop=True, inplace=True)
normalized_tracks = pd.json_normalize(exploded_train['tracks'])
train_playlist_songs = pd.concat([exploded_train.drop(columns='tracks'), normalized_tracks], axis=1)

In [27]:
train_playlist_songs

,name,collaborative,pid,modified_at,num_tracks,num_albums,num_followers,num_edits,duration_ms,num_artists,description,pos,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms,album_name
0,Throwbacks,false,0,1493424000,52,47,1,6,11532414,37,NaN,0,Missy Elliott,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,spotify:artist:2wIVse2owClT7go1WT98tk,Lose Control (feat. Ciara & Fat Man Scoop),spotify:album:6vV5UrXcfyQD1wu4Qo2I9K,226863,The Cookbook
1,Throwbacks,false,0,1493424000,52,47,1,6,11532414,37,NaN,1,Britney Spears,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,spotify:artist:26dSoYclwsYLMAKD3tpOr4,Toxic,spotify:album:0z7pVBGOD7HCIB7S8eLkLI,198800,In The Zone
2,Throwbacks,false,0,1493424000,52,47,1,6,11532414,37,NaN,2,Beyoncé,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,spotify:artist:6vWDO969PvNqNYHIOW5v0m,Crazy In Love,spotify:album:25hVFAxTlDvXbx2X2QkUkE,235933,Dangerously In Love (Alben für die Ewigkeit)
3,Throwbacks,false,0,1493424000,52,47,1,6,11532414,37,NaN,3,Justin Timberlake,spotify:track:1AWQoqb9bSvzTjaLralEkT,spotify:artist:31TPClRtHm23RisEBtV3X7,Rock Your Body,spotify:album:6QPkyl04rXwTGlGlcYaRoW,267266,Justified
4,Throwbacks,false,0,1493424000,52,47,1,6,11532414,37,NaN,4,Shaggy,spotify:track:1lzr43nnXAijIGYnCT8M8H,spotify:artist:5EvFsr3kj42KNv97ZEnqij,It Wasn't Me,spotify:album:6NmFmPX56pcLBOFMhIiKvF,227600,Hot Shot
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67498,thinking of you,false,999,1507766400,44,43,1,34,9065801,37,NaN,39,James Arthur,spotify:track:5uCax9HTNlzGybIStD3vDh,spotify:artist:4IWBUUAFIplrNtaOHcJPRM,Say You Won't Let Go,spotify:album:7oiJYvEJHsmYtrgviAVIBD,211466,Back from the Edge
67499,thinking of you,false,999,1507766400,44,43,1,34,9065801,37,NaN,40,Big Words,spotify:track:0P1oO2gREMYUCoOkzYAyFu,spotify:artist:0sHN89qak07mnug3LVVjzP,The Answer,spotify:album:5jrsRHRAmetu5e7RRBoxj7,263679,"Hollywood, a Beautiful Coincidence"
67500,thinking of you,false,999,1507766400,44,43,1,34,9065801,37,NaN,41,Allan Rayman,spotify:track:2oM4BuruDnEvk59IvIXCwn,spotify:artist:6Yv6OBXD6ZQakEljaGaDAk,25.22,spotify:album:3CbNgBzI7r9o0F6VjH9sTY,189213,Roadhouse 01
67501,thinking of you,false,999,1507766400,44,43,1,34,9065801,37,NaN,42,Jon Jason,spotify:track:4Ri5TTUgjM96tbQZd5Ua7V,spotify:artist:77bNdkKYBBmc30CisCA6tE,Good Feeling,spotify:album:2dZ7oVNQBeLlpoUYfbEsJP,194720,Good Feeling


In [28]:
#train_playlist_songs.columns.tolist()

In [29]:
# Discovering of data
#train_playlist_songs.isnull().sum()

### Take some playlist as user's preferencies / TODO Temporary solution 

In [30]:
# Take un example for user's playlist
def load2(i):
    start = i*10000
    path = "data/mpd.slice." + str(start) + "-" + str(start+999) + ".json"
    data = json.load(open(path,'r'))
    return pd.DataFrame.from_dict(data['playlists'], orient='columns')

user_PL = []
user_PL.append(load2(1))
df_user_PL = pd.concat(user_PL)
df_user_PL = df_user_PL.iloc[1]
df_user_PL

name                                                     Childhood
collaborative                                                false
pid                                                          10001
modified_at                                             1412985600
num_tracks                                                      28
num_albums                                                      19
num_followers                                                    1
tracks           [{'pos': 0, 'artist_name': 'The Killers', 'tra...
num_edits                                                        3
duration_ms                                                5823244
num_artists                                                     14
description                                                    NaN
Name: 1, dtype: object

#### Select all playlists according track_uri from user's playlist

In [31]:
# URI's of user's prefered tracks as list

pref_tracksURI_list = [el['track_uri'] for el in df_user_PL['tracks']]
print(len(pref_tracksURI_list))
pref_tracksURI_list

28


['spotify:track:7oK9VyNzrYvRFo7nQEYkWN',
 'spotify:track:4BodkdQk2Ouo6BFbLqoWtG',
 'spotify:track:37LNThlFJaNmi9SeLrv8H6',
 'spotify:track:4dYeLvp1VhtMRBQvSUScoz',
 'spotify:track:7I1P7Dam9l2fhMRMkshH16',
 'spotify:track:6TlRNJaezOdzdECnQeRuMM',
 'spotify:track:3iL2l5gUqyPS6vDwJFgJTR',
 'spotify:track:3RBGG1be9aBaSvjnw6ABkh',
 'spotify:track:2WOiei9edVwlABd8cJfzja',
 'spotify:track:3LHMFgMolEbhhGBC47OuAi',
 'spotify:track:0gdLTqxAY4DDUQxXzmwj1z',
 'spotify:track:7GCaZax7ExKSNYFv8eQCvL',
 'spotify:track:2l57cfmCnOkwNX1tky02n1',
 'spotify:track:1FMHNVeJ9s1x1l1WlaRs2I',
 'spotify:track:4jVhSbMMHctghoOTFuLPB2',
 'spotify:track:5lDriBxJd22IhOH9zTcFrV',
 'spotify:track:5Ozo0lsCaVNJPlKC7RtDnp',
 'spotify:track:2Eh5ro0vWcZQY7r1YeYqxc',
 'spotify:track:1hIupFeRu3nmMcjbjxPnMc',
 'spotify:track:714Lw0m2SmCEhKSPw0Dn8J',
 'spotify:track:23oxJmDc1V9uLUSmN2LIvx',
 'spotify:track:48O6kz322Dzu1R6Al5147q',
 'spotify:track:26QSvu9Eu82RKCgPfuZqlk',
 'spotify:track:5fGkcxOENUG2TX16Ij26oz',
 'spotify:track:

# User-based recommendation (based on playlist)

In [32]:
# Select playlists contained user's preffered tracks
pref_in_train = train_playlist_songs.loc[train_playlist_songs['track_uri'].isin(pref_tracksURI_list)]
pref_in_train

# Range them according to relevance
most_relevant = pref_in_train['pid'].value_counts().rename_axis('pid').reset_index(name='Frequency')
most_relevant

,pid,Frequency
0,844,5
1,987,4
2,45,4
3,921,3
4,895,3
...,...,...
58,565,1
59,601,1
60,625,1
61,642,1


In [33]:
# Select 500 tracks from the most relevant lists
pids = most_relevant['pid'].tolist()
ret = []
total = 0
playlists = 0

for pid in pids:
    if (total < 500):
        playlists += 1
        print(train.loc[train['pid'] == pid]['num_tracks']) #TODO for debug, to sut after
        
        tracks_series = train.loc[train['pid'] == pid]['tracks']
        for ind, list_tracks in tracks_series.items():
            for tr in list_tracks:
                ret.append(tr)
                total += 1

print('\n{} first playlists was used'.format(playlists))
print('\n{} tracks was added'.format(total))
print('\nLength of obtained list = {}\n'.format(len(ret)))

recommended = ret[:500]
recommended

844    150
Name: num_tracks, dtype: int64
987    172
Name: num_tracks, dtype: int64
45    76
Name: num_tracks, dtype: int64
921    112
Name: num_tracks, dtype: int64

4 first playlists was used

510 tracks was added

Length of obtained list = 510



[{'pos': 0,
  'artist_name': 'The Strokes',
  'track_uri': 'spotify:track:3dJAKrILAnNJwKFAzNmJvZ',
  'artist_uri': 'spotify:artist:0epOFNiUfyON9EYx7Tpr6V',
  'track_name': 'Automatic Stop',
  'album_uri': 'spotify:album:5wwZ43m3GAOfdWIxkKHpUz',
  'duration_ms': 200386,
  'album_name': 'Room On Fire'},
 {'pos': 1,
  'artist_name': 'Duffy',
  'track_uri': 'spotify:track:1zusIxNqJu8i4g6P6hJ2Qa',
  'artist_uri': 'spotify:artist:37NqXwtb6nIEqRt4EJSoIO',
  'track_name': 'Mercy',
  'album_uri': 'spotify:album:6Yw7f0kqQ1gbt2OHa7fi1g',
  'duration_ms': 219800,
  'album_name': 'Rockferry'},
 {'pos': 2,
  'artist_name': 'Taking Back Sunday',
  'track_uri': 'spotify:track:6PsISpFaCEKwysPfwO9SfC',
  'artist_uri': 'spotify:artist:24XtlMhEMNdi822vi0MhY1',
  'track_name': "Cute Without The 'E' (Cut From The Team)",
  'album_uri': 'spotify:album:3V0GctkS3kbM50UY6W80hN',
  'duration_ms': 211546,
  'album_name': 'Tell All Your Friends'},
 {'pos': 3,
  'artist_name': 'Taking Back Sunday',
  'track_uri': '

# Item-based recommendation (based on tracks)